In [19]:
# import sys
# sys.path.append('../../src')
import re
import pathlib
import src.data_process.neg_sample as ng_sample
import pandas as pd 
from sentence_transformers import SentenceTransformer
import torch
from src.utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL
from hydra import initialize, compose




In [20]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [21]:
with initialize(version_base=None, config_path="../src/conf"):
    cfg = compose(config_name="config", overrides=[])

In [22]:
# NEG_TRAIN = 4

In [23]:
if device.type == 'cpu':
    use_amp=False
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test)).iloc[:202,]
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).iloc[:100,].reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).iloc[:100*cfg.params.neg_train,].reset_index(drop=True)
else:
    use_amp=True
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test))
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)

In [24]:
df_train_pos[DEFAULT_RATING_COL] = 1

In [25]:
def concat_index(df1, df2):
    df2.index = df2.index//cfg.params.neg_train
    return pd.concat([df1, df2], axis=0).sort_index(kind='mregesort').reset_index(drop=True)

In [28]:
df_train_all = concat_index(df_train_pos, df_train_neg)

In [29]:
df_train_all['flag'] = 1
df_test_ori['flag'] = 0
df_all = pd.concat([df_train_all, df_test_ori], axis=0).reset_index(drop=True)

In [30]:
df_all.head()

,userid,itemid,rating,flag
0,7,309823,1,1
1,7,516130,0,1
2,7,544280,0,1
3,7,372893,0,1
4,7,954871,0,1


# Job

In [31]:
jobs = pd.read_csv('../data/jobs/tsv/jobs.tsv', sep='\t', encoding='utf-8', on_bad_lines='skip')

jobs.rename(columns = {'JobID':"item_id", 'City':'City_job',  'State':'State_job', 'Country':'Country_job',                       'Zip5':'Zip_job'}, inplace=True)

jobs['Title'] = jobs['Title'].fillna('')
 

/tmp/ipykernel_7804/1338635980.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv('../data/jobs/tsv/jobs.tsv', sep='\t', encoding='utf-8', on_bad_lines='skip')


In [48]:
jobs.head()

,item_id,WindowID,Title,Description,Requirements,City_job,State_job,Country_job,Zip_job,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59


In [ ]:
jobs_sub[]

In [32]:
jobs_sub = jobs[jobs.item_id.isin(df_all.itemid)]

In [50]:
sum(jobs['item_id']==154928)

0

In [49]:
jobs[jobs['item_id']==154928]

,item_id,WindowID,Title,Description,Requirements,City_job,State_job,Country_job,Zip_job,StartDate,EndDate


In [12]:
def preprocessor(text):
    text = text.replace('\\r', '').replace('&nbsp', '').replace('\n', '')
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

jobs_sub['Title'] = jobs_sub['Title'].astype(dtype='str').apply(preprocessor)



/tmp/ipykernel_47727/973742996.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_sub['Title'] = jobs_sub['Title'].astype(dtype='str').apply(preprocessor)


In [20]:
 
sbert_model =  SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device='cuda')


In [22]:
jobs_sub["Title_em"]=  jobs_sub['Title'].apply(lambda x : sbert_model.encode(x))


/tmp/ipykernel_47727/3605969977.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_sub["Title_em"]=  jobs_sub['Title'].apply(lambda x : sbert_model.encode(x))


In [23]:
jobs_sub

,item_id,WindowID,Title,Description,Requirements,City_job,State_job,Country_job,Zip_job,StartDate,EndDate,Title_em
440254,76,3,punch out foreman,10 yrs exp minimum for underground utilities c...,Please refer to the Job Description to view th...,Oviedo,FL,US,32765.0,2012-03-29 01:44:30.227,2012-04-28 23:59:59,"[0.019618202, 0.09344988, 0.02695138, 0.002974..."
440256,85,3,qa qc senior assistant,This position provides support in the quality ...,Please refer to the Job Description to view th...,Eustis,FL,US,32726.0,2012-03-30 01:23:58.62,2012-04-29 23:59:59,"[0.011078028, 0.019362302, -0.016758027, 0.008..."
440259,98,3,furniture craftsman,<b> <b>FURNITURE CRAFTSMAN</b> </b> —— Ex...,Please refer to the Job Description to view th...,Orlando,FL,US,32801.0,2012-04-05 12:52:07.437,2012-05-04 23:59:59,"[-0.004492085, 0.03667915, 0.010740442, 0.0155..."
440261,100,3,marketing homecare,We are looking for someone with experience in ...,Please refer to the Job Description to view th...,Orlando,FL,US,32803.0,2012-04-08 01:27:00.893,2012-05-07 23:59:59,"[0.012990964, -0.018705405, -0.04509775, -0.03..."
440266,114,3,leasing consultant,<b> <b> LEASING CONSULTANT </b> </b> —— F/...,Please refer to the Job Description to view th...,Orlando,FL,US,32817.0,2012-04-13 12:06:14.227,2012-05-12 23:59:59,"[0.0058157262, 0.012847453, -9.123508e-05, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...
861360,1116240,5,retail management professional,<SPAN>\r<HR>\r<P dir=ltr align=center><STRONG>...,<SPAN>\r<HR>\r<P dir=ltr align=center><STRONG>...,New York,NY,US,NaN,2012-04-27 15:54:44.637,2012-05-26 23:59:59,"[0.012847574, -0.015192667, -0.046453502, -0.0..."
861364,1116257,5,senior compliance analyst,<p>Located in the vibrant Harbor East section ...,<p><strong>Experience and Education Requiremen...,Baltimore,MD,US,21202.0,2012-05-15 13:44:38.507,2012-06-14 23:59:00,"[0.012384181, 0.013370257, -0.012150419, -0.03..."
861366,1116283,5,cnc mill machinists and cnc operators,<span>CNC Mill Machinists and CNC operators /...,<p><b>CNC Mill Machinists and CNC operators /...,Montgomeryville,PA,US,18936.0,2012-05-02 10:08:25.743,2012-06-01 23:59:59,"[-0.045443065, -0.0056769284, -0.037670527, 0...."
861369,1116295,5,operations expert,"FRED&rsquo;S Stores, America&rsquo;s favorite ...",<br>\r*&nbsp; The successful candidate will ha...,Calhoun City,MS,US,NaN,2012-05-29 16:43:22.227,2012-06-28 23:59:00,"[0.03150434, 0.017827582, -0.021305485, -0.024..."


In [25]:
jobs_sub.to_csv("../../data/jobs/job_title_embed.csv")

In [26]:
jobs_sub[['item_id', 'Title','Title_em']].to_csv("../../data/jobs/job_title_embed2.csv")

In [30]:
jobs_sub[['item_id','Title_em']].reset_index().to_feather("../../data/jobs/job_title_embed3")